## Experiment-06
### Amirreza Fosoul and Bithiah Yuan

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import string
import csv
import re
import time
from pyspark.sql.functions import *
from pyspark.sql.functions import split
import pyspark.sql.functions as f
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.functions import desc
from pyspark.sql.window import Window as W
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import concat, col, lit, size
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
sc = spark.sparkContext
import random
spark = SparkSession.builder.appName('ex6').getOrCreate()

In [5]:
# Read user ratings into Dataframe
user_df = spark.read.option("delimiter", ";").csv('./example0.txt')
#user_df = spark.read.option("delimiter", ";").csv('./example.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID"))

user_pre = user_df

user_df = user_df.select("userID", f.split("paperID", ",").alias("papers"), f.explode(f.split("paperID", ",")).alias("paperID"))
user_df = user_df.drop("papers")

# Count the number of distinct users
numUsers = user_df.select("userID").distinct().count()

# Get a dataframe of the distinct papers
d_paper = user_df.select("paperID").distinct()

user_pre.show()

+--------------------+--------------------+
|              userID|             paperID|
+--------------------+--------------------+
|28d3f81251d94b097...|3929762,503574,58...|
|d0c9aaa788153daea...|2080631,6343346,5...|
|f05bcffe7951de9e5...|1158654,478707,12...|
|ca4f1ba4094011d9a...|              278019|
|d1d41a15201915503...|6610569,6493797,6...|
|f2f77383828ea6d39...|943458,238121,763429|
|9c883d02115400f7b...|3509971,3509965,2...|
|b656009a6efdc8b1a...|771870,181369,121...|
|cf9c7f356092c34be...|               90558|
|0f5cbb39410a9278f...|             9344598|
|d85f7d83f27b3f533...|7610843,3633347,4...|
|586c867a0688250ac...|464760,466011,373...|
|10fdfaf945d5c27ad...|             2010550|
|589b870a611c25fa9...|1283233,1305474,1...|
|90f1a3e6fcdbf9bc5...|115945,11733005,9...|
|7e070a9da96672e05...|             1071959|
|3b715ebaf1f8f81a1...|4119394,3378798,1...|
|488fb15e8c77f8054...|1523301,5281566,5...|
|3fdf355e59949c79d...|7077220,1289842,1...|
|c6b59086a0bbac141...|2230995,30

### Exercise 6. 1 (Problem Modeling)

In [6]:
# Create a user hash map dataframe by adding a column with descending row numbers 
# to the pre-explode user dataframe
# user_hash_map = user_pre.withColumn("user_idx", F.monotonically_increasing_id().cast(IntegerType()))

# Select the user indices and paperID
#rating = user_hash_map.select('user_idx', 'paperID')
rating = user_pre.select('userID', 'paperID')

# Split the string if all paperIDs into individual paperIDs and cast the datatype to int
rating = rating.withColumn("papers", split(col("paperID"), ",\s*").cast(ArrayType(IntegerType())).alias("papers"))
# Drop the column with the string of paperIDs
rating = rating.drop("paperID")

# Add a column of 1's called ratings for the rated papers w.r.t to the user index
rating = rating.withColumn("rating", lit(1))

# Transform the distinct paperIDs dataframe to a list
paper_list = list(d_paper.select('paperID').toPandas()['paperID'])
# Map each distinct paper into int
paper_list = list(map(int, paper_list))

# Function to call in udf
def unrated(papers):
    # Transform the list of distinct papers and the list of rated papers of each user to a set
    # Substract the two sets to get the list of unrated papers for each user
    # Transform back to list
    unrated = list(set(paper_list) - set(papers))

    # Select n random papers from the unrated papers where n is the 
    # length of the rated papers list of each user
    unrated_papers = random.sample(unrated, len(papers))
            
    return unrated_papers


# udf to get a list of unrated papers with the length of rated papers for each user
get_unrated = udf(lambda x: unrated(x), ArrayType(IntegerType()))

# Add a new column of unrated papers for each user
rating = rating.withColumn("unrated", get_unrated(rating.papers))

# Create a dataframe with the rated papers and their ratings of 1's
rated_df = rating.select("userID", "papers", "rating")

# Create a dataframe with the unrated papers
unrated_df = rating.select("userID", col("unrated").alias("papers"))
# Add a column of ratings of 0's to the unrated papers
unrated_df = unrated_df.withColumn("rating", lit(0))

# Union the rated and unrated dataframes and order by the user indices
rating_matrix = rated_df.union(unrated_df).orderBy("userID")

# Explode the lists of paperIDs w.r.t. each user
rating_matrix = rating_matrix.select("userID", f.explode("papers").alias("paperID"), 'rating')

rating_matrix.show()

+--------------------+-------+------+
|              userID|paperID|rating|
+--------------------+-------+------+
|01ecfc66d01b0dd84...| 970908|     1|
|01ecfc66d01b0dd84...|  84168|     1|
|01ecfc66d01b0dd84...|1843498|     1|
|01ecfc66d01b0dd84...| 808689|     1|
|01ecfc66d01b0dd84...| 508086|     1|
|01ecfc66d01b0dd84...| 397003|     1|
|01ecfc66d01b0dd84...| 841861|     0|
|01ecfc66d01b0dd84...|2242776|     0|
|01ecfc66d01b0dd84...| 878302|     0|
|01ecfc66d01b0dd84...|1172205|     0|
|01ecfc66d01b0dd84...| 151946|     0|
|01ecfc66d01b0dd84...|2109140|     0|
|0ad6516296d95068c...|2734645|     1|
|0ad6516296d95068c...|1218426|     1|
|0ad6516296d95068c...| 748113|     1|
|0ad6516296d95068c...| 179912|     1|
|0ad6516296d95068c...|2069872|     1|
|0ad6516296d95068c...| 359059|     1|
|0ad6516296d95068c...| 763543|     1|
|0ad6516296d95068c...|2967998|     1|
+--------------------+-------+------+
only showing top 20 rows

